In [ ]:
# Policy Gradient

# https://github.com/seungeunrho/minimalRL/blob/master/REINFORCE.py
# Ref: https://www.youtube.com/watch?v=y8UPGr36ccI (11分開始有說明loss)

In [1]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

#Hyperparameters
learning_rate = 0.0002
gamma         = 0.98

class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.data = []
        
        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 2)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=0)
        return x
      
    def put_data(self, item):
        self.data.append(item)
        
    def train_net(self):
        R = 0
        # data是玩完一個episode蒐集起來的(r, a)pair
        for r, log_prob in self.data[::-1]:
            R = r + gamma * R  # 除了看instant r，也要考慮未來的期望衰減reward R，越前面的r越會考慮未來R
            # 這裡就等於max(log_prob) --> cross entropy，min -summation(y_hat * log(y_i)) --> 因為y_hat大部分都是0 --> = max(log(y_i))
            loss = -log_prob * R  # 這邊就像train一個分類問題，把s跟a對起來訓練，然後給予R的權重，Reward越大代表這個a答案越好，權重越大
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()  # 每一步都更新一次
        self.data = []

def main():
    env = gym.make('CartPole-v1')
    pi = Policy()
    score = 0.0
    print_interval = 20
    
    for n_epi in range(10000):
        s = env.reset()
        for t in range(501): # CartPole-v1 forced to terminates at 500 step.
            # print(s)  # [-0.05109565  0.01887992  0.10848381  0.265779  ]
            prob = pi(torch.from_numpy(s).float())  # [0.45, 0.55]
            m = Categorical(prob)  # choose action based on the prob
            a = m.sample()
            s_prime, r, done, info = env.step(a.item())
            pi.put_data((r,torch.log(prob[a])))
            
            s = s_prime
            score += r
            if done:
                break

        pi.train_net()
        
        if n_epi % print_interval==0 and n_epi!=0:
            print("# of episode :{}, avg score : {}".format(n_epi, score/print_interval))
            score = 0.0
    env.close()
    
if __name__ == '__main__':
    main()

[-0.01778966 -0.02186928 -0.03338169 -0.01818751]
[-0.01822705 -0.216497   -0.03374544  0.26377902]
[-0.02255699 -0.02091004 -0.02846986 -0.03935367]
[-0.02297519  0.17460836 -0.02925693 -0.34088142]
[-0.01948302 -0.02008537 -0.03607456 -0.05756612]
[-0.01988473 -0.214672   -0.03722588  0.22352031]
[-0.02417817 -0.01903831 -0.03275547 -0.0806688 ]
[-0.02455894 -0.21367577 -0.03436885  0.20150239]
[-0.02883245 -0.40828976 -0.0303388   0.48314852]
[-0.03699825 -0.60297068 -0.02067583  0.76611727]
[-0.04905766 -0.40757024 -0.00535349  0.467001  ]
[-0.05720906 -0.60261614  0.00398653  0.75799175]
[-0.06926139 -0.40754936  0.01914637  0.46656593]
[-0.07741237 -0.21270308  0.02847769  0.17997878]
[-0.08166644 -0.01799997  0.03207726 -0.10358616]
[-0.08202644  0.17664794  0.03000554 -0.38597873]
[-0.07849348  0.37133137  0.02228597 -0.66905212]
[-0.07106685  0.56613647  0.00890492 -0.95463567]
[-0.05974412  0.7611375  -0.01018779 -1.24450761]
[-0.04452137  0.56614773 -0.03507794 -0.9550333 ]


[ 0.55632864  2.06562827 -0.13938316 -1.77095184]
[ 0.5976412   2.26202091 -0.1748022  -2.10352994]
[-0.02904868  0.03069599  0.04746917  0.00366194]
[-0.02843476 -0.16507346  0.04754241  0.31093603]
[-0.03173623 -0.36083938  0.05376113  0.6182253 ]
[-0.03895302 -0.55666945  0.06612563  0.92734417]
[-0.0500864  -0.36249967  0.08467252  0.65615281]
[-0.0573364  -0.16865217  0.09779557  0.39128726]
[-0.06070944  0.0249557   0.10562132  0.13096918]
[-0.06021033  0.21841856  0.1082407  -0.12661296]
[-0.05584196  0.02192585  0.10570844  0.19816185]
[-0.05540344  0.21538951  0.10967168 -0.05939334]
[-0.05109565  0.01887992  0.10848381  0.265779  ]
[-0.05071805  0.21229978  0.11379939  0.00918648]
[-0.04647205  0.01574529  0.11398312  0.33549604]
[-0.04615715 -0.18079869  0.12069304  0.66183747]
[-0.04977312  0.01245557  0.13392979  0.40946251]
[-0.04952401  0.20544971  0.14211904  0.1618215 ]
[-0.04541502  0.00860947  0.14535547  0.49574755]
[-0.04524283  0.2014148   0.15527042  0.25217359]


[-0.07885978 -0.18760084  0.0317372   0.26847341]
[-0.08261179 -0.38316101  0.03710667  0.57099511]
[-0.09027501 -0.18857853  0.04852657  0.29022916]
[-0.09404659  0.00581909  0.05433116  0.01323721]
[-0.0939302   0.20012147  0.0545959  -0.26182079]
[-0.08992777  0.39442331  0.04935949 -0.53679606]
[-0.08203931  0.58881776  0.03862356 -0.81352681]
[-0.07026295  0.39318872  0.02235303 -0.50894959]
[-0.06239918  0.19775909  0.01217404 -0.20930714]
[-0.058444    0.0024652   0.00798789  0.08719107]
[-0.05839469 -0.19277033  0.00973172  0.38238344]
[-0.0622501   0.0022121   0.01737938  0.09278472]
[-0.06220586 -0.19315458  0.01923508  0.39089978]
[-0.06606895 -0.38854417  0.02705307  0.6895847 ]
[-0.07383983 -0.19380789  0.04084477  0.4055398 ]
[-0.07771599  0.00071176  0.04895556  0.1260087 ]
[-0.07770175 -0.1950761   0.05147574  0.43372582]
[-0.08160328 -0.39088761  0.06015025  0.74218122]
[-0.08942103 -0.19664528  0.07499388  0.46901816]
[-0.09335393 -0.3927421   0.08437424  0.78436417]


[ 0.01514007  0.0120294  -0.04696142 -0.04455468]
[ 0.01538066  0.2077922  -0.04785251 -0.35167672]
[ 0.0195365   0.01338226 -0.05488605 -0.07445889]
[ 0.01980415 -0.18091164 -0.05637522  0.20041469]
[ 0.01618591  0.01496939 -0.05236693 -0.10950561]
[ 0.0164853  -0.17936457 -0.05455704  0.16620689]
[ 0.01289801  0.01649418 -0.05123291 -0.14317593]
[ 0.01322789  0.212311   -0.05409642 -0.45157164]
[ 0.01747411  0.40815461 -0.06312786 -0.76080391]
[ 0.02563721  0.60408682 -0.07834393 -1.07266408]
[ 0.03771894  0.41008294 -0.09979722 -0.80556043]
[ 0.0459206   0.2164603  -0.11590842 -0.54586326]
[ 0.05024981  0.41300373 -0.12682569 -0.87270106]
[ 0.05850988  0.21981338 -0.14427971 -0.62242985]
[ 0.06290615  0.41662387 -0.15672831 -0.95684944]
[ 0.07123863  0.61346658 -0.1758653  -1.29438423]
[ 0.08350796  0.4209603  -0.20175298 -1.06151707]
[ 0.0162365  -0.02013043  0.00443618  0.0289072 ]
[ 0.01583389  0.17492762  0.00501433 -0.26237277]
[ 0.01933244 -0.02026555 -0.00023313  0.03188751]


[ 0.01167288  0.3396481   0.04335646 -0.41978696]
[ 0.01846585  0.14393949  0.03496073 -0.11375711]
[ 0.02134463  0.33854351  0.03268558 -0.3952084 ]
[ 0.02811551  0.5331868   0.02478141 -0.67740954]
[ 0.03877924  0.33772947  0.01123322 -0.37702858]
[ 0.04553383  0.14244979  0.00369265 -0.08082504]
[ 0.04838283 -0.0527249   0.00207615  0.21302064]
[ 0.04732833 -0.24787647  0.00633656  0.50635775]
[ 0.0423708  -0.05284438  0.01646372  0.21567843]
[ 0.04131391 -0.24819778  0.02077729  0.51350893]
[ 0.03634996 -0.44360611  0.03104747  0.81266624]
[ 0.02747783 -0.63913925  0.04730079  1.11495109]
[ 0.01469505 -0.83484919  0.06959981  1.42208891]
[-0.00200194 -0.6406537   0.09804159  1.15194674]
[-0.01481501 -0.8369084   0.12108053  1.47369318]
[-0.03155318 -0.64345659  0.15055439  1.22115069]
[-0.04442231 -0.45056064  0.1749774   0.97917576]
[-0.05343352 -0.64754147  0.19456092  1.32131655]
[ 0.02915209 -0.04721381  0.03858614 -0.04665125]
[ 0.02820781 -0.24286721  0.03765311  0.25795191]


...

[ 0.03823652  0.04884405 -0.00492815 -0.00813872]
[ 0.0392134  -0.14620688 -0.00509092  0.28298525]
[ 3.62892640e-02 -3.41255850e-01  5.68782630e-04  5.74058191e-01]
[ 0.02946415 -0.14614188  0.01204995  0.2815545 ]
[ 0.02654131  0.04880614  0.01768104 -0.00730369]
[ 0.02751743  0.24367011  0.01753496 -0.29435602]
[ 0.03239083  0.43853774  0.01164784 -0.5814575 ]
[ 4.11615892e-02  2.43254535e-01  1.86923306e-05 -2.85128178e-01]
[ 0.04602668  0.04813232 -0.00568387  0.00756064]
[ 0.04698933  0.24333532 -0.00553266 -0.28691018]
[ 0.05185603  0.43853573 -0.01127086 -0.58133291]
[ 0.06062675  0.63381378 -0.02289752 -0.87754494]
[ 0.07330302  0.43901037 -0.04044842 -0.59214771]
[ 0.08208323  0.24447732 -0.05229137 -0.31247539]
[ 0.08697278  0.44030371 -0.05854088 -0.62118014]
[ 0.09577885  0.24604604 -0.07096448 -0.34749397]
[ 0.10069977  0.44210184 -0.07791436 -0.66168353]
[ 0.10954181  0.24814551 -0.09114803 -0.39451532]
[ 0.11450472  0.05442714 -0.09903834 -0.13190442]
[ 0.11559326  0.25

[ 0.05204919  0.4522392  -0.14055567 -0.90119995]
[ 0.06109398  0.25927269 -0.15857967 -0.65579208]
[ 0.06627943  0.06667224 -0.17169551 -0.41694364]
[ 0.06761287 -0.12565364 -0.18003439 -0.18292745]
[ 0.0650998  -0.3178041  -0.18369294  0.04799517]
[ 0.05874372 -0.12058847 -0.18273303 -0.29655381]
[ 0.05633195  0.07660414 -0.18866411 -0.64084308]
[ 0.05786403 -0.11545668 -0.20148097 -0.41300263]
[-0.0479618  -0.03973864 -0.04920241 -0.02342192]
[-0.04875657  0.15605312 -0.04967085 -0.33121383]
[-0.04563551 -0.0383279  -0.05629512 -0.05459913]
[-0.04640207  0.15755417 -0.05738711 -0.36449879]
[-0.04325099 -0.03670725 -0.06467708 -0.0904489 ]
[-0.04398513 -0.2308454  -0.06648606  0.18114765]
[-0.04860204 -0.03483818 -0.06286311 -0.1317461 ]
[-0.0492988  -0.22900595 -0.06549803  0.14046067]
[-0.05387892 -0.42313166 -0.06268882  0.41178221]
[-0.06234155 -0.61731151 -0.05445317  0.68406141]
[-0.07468778 -0.42147746 -0.04077194  0.3747444 ]
[-0.08311733 -0.61599726 -0.03327706  0.654298  ]


[-0.03100542  0.23593468  0.0487939  -0.2517321 ]
[-0.02628672  0.04015116  0.04375926  0.05593337]
[-0.0254837   0.23461924  0.04487793 -0.22262839]
[-0.02079132  0.03888553  0.04042536  0.08386624]
[-0.02001361  0.23340539  0.04210269 -0.19579334]
[-0.0153455   0.03770728  0.03818682  0.10986822]
[-0.01459135  0.23226181  0.04038418 -0.17052662]
[-0.00994612  0.42678317  0.03697365 -0.45020104]
[-0.00141045  0.6213632   0.02796963 -0.73100383]
[ 0.01101681  0.81608767  0.01334955 -1.0147543 ]
[ 0.02733856  0.62079025 -0.00694553 -0.71790961]
[ 0.03975437  0.4257651  -0.02130372 -0.42742091]
[ 0.04826967  0.23095125 -0.02985214 -0.14152927]
[ 0.0528887   0.03626928 -0.03268273  0.14158835]
[ 0.05361408  0.23184369 -0.02985096 -0.16122362]
[ 0.05825096  0.03716153 -0.03307543  0.12189465]
[ 0.05899419  0.23274135 -0.03063754 -0.18103703]
[ 0.06364901  0.0380709  -0.03425828  0.10182558]
[ 0.06441043  0.23366664 -0.03222177 -0.20146585]
[ 0.06908376  0.42923426 -0.03625109 -0.50413655]


[ 0.01482107 -0.04413115 -0.02542402 -0.02471157]
[ 0.01393844 -0.23887945 -0.02591826  0.25984261]
[ 0.00916085 -0.04339727 -0.0207214  -0.04090129]
[ 0.00829291 -0.23821604 -0.02153943  0.24517248]
[ 0.00352859 -0.04279318 -0.01663598 -0.05422593]
[ 0.00267272  0.15256331 -0.0177205  -0.35211086]
[ 0.00572399 -0.04230222 -0.02476272 -0.06506801]
[ 0.00487795 -0.23706055 -0.02606408  0.21970043]
[ 0.00013673 -0.04157591 -0.02167007 -0.08108891]
[-0.00069478 -0.23638063 -0.02329185  0.20467906]
[-0.0054224  -0.43116188 -0.01919826  0.48992451]
[-0.01404563 -0.23577443 -0.00939977  0.1912534 ]
[-0.01876112 -0.04051928 -0.00557471 -0.10437988]
[-0.01957151 -0.2355609  -0.0076623   0.18653904]
[-0.02428273 -0.04033016 -0.00393152 -0.10855116]
[-0.02508933 -0.23539555 -0.00610255  0.1828888 ]
[-0.02979724 -0.43042966 -0.00244477  0.47364037]
[-0.03840583 -0.23527326  0.00702804  0.18018789]
[-0.0431113  -0.04025258  0.0106318  -0.11026968]
[-0.04391635  0.15471542  0.0084264  -0.39957946]


KeyboardInterrupt: 